In [1]:
!pip install "fsspec>=2023.1.0,<2025.3.1"
!pip install datasets
!pip install transformers
!pip install librosa
!pip install jiwer
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 5.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.6.0+cu124 requires nvid

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import torch
from datasets import load_dataset, DatasetDict
from transformers import (
    Wav2Vec2Processor,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2ForCTC,
    Wav2Vec2CTCTokenizer,
    TrainingArguments,
    Trainer,
    Wav2Vec2Config,
    get_scheduler
)
from evaluate import load
from dataclasses import dataclass, field

import os, json
import numpy as np
from typing import Any, Dict, List, Optional, Union

2025-05-23 12:13:17.755365: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748002397.936043      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748002397.986489      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [16]:
# Configuration
DATA_DIR = "/kaggle/input/wav2vec2-0pfinetune/Dataset"
CHECKPOINT_DIR = "/kaggle/working/checkpoints"
CHECKPOINT_PREFIX = f"{CHECKPOINT_DIR}/w2v"
LOAD_CONFIG_PATH = "load.config"
PROCESSOR_PATH = "/kaggle/working/processor"  # Path to save/load processor
TOKENIZER_PATH = "/kaggle/working/tokenizer"  # Path to save/load tokenizer

MODEL_NAME = "/kaggle/input/wav2vec-demo-model2"  # Base architecture | Please change it bro!

BENGALI_TOKENIZER = "/kaggle/input/tanmoyiowav2vec2-large-xlsr-bengali"  # Bengali tokenizer
BATCH_SIZE = 16
GRADIENT_ACCUMULATION_STEPS = 2
NUM_EPOCHS_PER_CHUNK = 3

# Create necessary directories
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(PROCESSOR_PATH, exist_ok=True)
os.makedirs(TOKENIZER_PATH, exist_ok=True)

In [4]:
# Configuration
DATA_DIR = "/kaggle/input/wav2vec2-0pfinetune/Dataset"
CHECKPOINT_DIR = "/kaggle/input/checkpoints"
CHECKPOINT_PREFIX = f"{CHECKPOINT_DIR}/w2v"
LOAD_CONFIG_PATH = "/kaggle/input/checkpoints/load.config"
PROCESSOR_PATH = "/kaggle/working/processor"  # Path to save/load processor
TOKENIZER_PATH = "/kaggle/working/tokenizer"  # Path to save/load tokenizer

MODEL_NAME = "/kaggle/input/wav2vec-demo-model2"  # Base architecture | Please change it bro!

BENGALI_TOKENIZER = "/kaggle/input/tanmoyiowav2vec2-large-xlsr-bengali"  # Bengali tokenizer
BATCH_SIZE = 16
GRADIENT_ACCUMULATION_STEPS = 2
NUM_EPOCHS_PER_CHUNK = 3

# Create necessary directories
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(PROCESSOR_PATH, exist_ok=True)
os.makedirs(TOKENIZER_PATH, exist_ok=True)

In [ ]:
# processed_dataset_chunk = load_dataset("parquet", data_files="/content/drive/MyDrive/dataset_chunk_0.parquet")

1. Determine iteration and Load data chunk

In [92]:
if os.path.exists(LOAD_CONFIG_PATH):
    with open(LOAD_CONFIG_PATH, "r") as f:
        iter_index = int(f.read().strip())
else:
    iter_index = 0

In [93]:

iter_index = 0
print(f"❇️ [LOAD INFO] Iteration Index: {iter_index}")

❇️ [LOAD INFO] Iteration Index: 0


2. Load data from chunks

In [94]:
# dataset = load_dataset("openslr", "SLR53", split="train[:4%]") # || Don't need the original dataset now. Therefore not loading it.

chunk_path = f"{DATA_DIR}/dataset_chunk_{iter_index}.parquet"
assert os.path.exists(chunk_path), f"Missing dataset chunk: {chunk_path}"
dataset = load_dataset("parquet", data_files={'train': chunk_path})
print(f"❇️ [LOAD INFO] Loaded data from {chunk_path}")

# Split into train and validation
train_valid_split = dataset['train'].train_test_split(
    test_size=187,  # Specify exact number for validation
    shuffle=True,   # Shuffle before splitting to randomize
    seed=42         # For reproducibility
)

# Rename splits to match your intention
dataset = DatasetDict({
    'train': train_valid_split['train'],
    'validation': train_valid_split['test']
})

train_dataset = dataset['train']
eval_dataset = dataset['validation']

Generating train split: 0 examples [00:00, ? examples/s]

❇️ [LOAD INFO] Loaded data from /kaggle/input/wav2vec2-0pfinetune/Dataset/dataset_chunk_0.parquet


In [ ]:
# data_len = processed_dataset_chunk['train'].num_rows

# prepared_dataset = DatasetDict()
# prepared_dataset['train'] = processed_dataset_chunk['train'].select((range(int(data_len * 0.8))))
# prepared_dataset['test'] = processed_dataset_chunk['train'].select((range(int(data_len * 0.8), data_len)))

# prepared_dataset

In [ ]:
# from datasets import load_dataset

# timit = load_dataset("parquet", data_files="dataset_raw_500.parquet")

# timit_len = timit['train'].num_rows

# timit['test'] = timit['train'].select((range(int(timit_len * 0.3))))
# timit['train'] = timit['train'].select((range(int(timit_len * 0.3), timit_len)))

# print(timit)

**3. Initialize or Load Processor, Model, and Training State**

In [95]:
if iter_index == 0 or not os.path.exists(f"{PROCESSOR_PATH}/preprocessor_config.json"):
    print("❇️ [LOAD INFO] Initializing new tokenizer and processor...")

    # Initialize the Bengali tokenizer
    tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
        BENGALI_TOKENIZER,
        unk_token="[UNK]",
        pad_token="[PAD]",
        word_delimiter_token="|"
    )

    # Save tokenizer for future use
    tokenizer.save_pretrained(TOKENIZER_PATH)

    # Initialize processor with the Bengali tokenizer
    processor = Wav2Vec2Processor.from_pretrained(
        BENGALI_TOKENIZER,
        tokenizer=tokenizer,
        feature_extractor_kwargs={'sampling_rate': 16000}
    )

    # Save processor for future use
    processor.save_pretrained(PROCESSOR_PATH)
else:
    print("❇️ [LOAD INFO] Loading existing tokenizer and processor...")
    tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(TOKENIZER_PATH)
    processor = Wav2Vec2Processor.from_pretrained(PROCESSOR_PATH)



# Setup model
if iter_index == 0:
    print("❇️ [LOAD INFO] Starting model training from scratch...")

    # Initialize model with appropriate config
    config = Wav2Vec2Config.from_pretrained(
        MODEL_NAME,
        vocab_size=len(processor.tokenizer),
        ctc_loss_reduction="mean",
        pad_token_id=processor.tokenizer.pad_token_id,
    )

    model = Wav2Vec2ForCTC.from_pretrained(
        MODEL_NAME,
        config=config,
        ignore_mismatched_sizes=True
    )

    # The faeture_extractor CNN was trained enough and should be freezed (according to the paper)
    model.freeze_feature_extractor()

    model.to("cuda")

    # Initialize optimizer and scheduler
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
    scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=100,
        num_training_steps=(NUM_EPOCHS_PER_CHUNK * (iter_index + 1)) * (len(train_dataset) // BATCH_SIZE)
    )

else:
    print(f"❇️ [LOAD INFO] Resuming from checkpoint {iter_index-1}...")

    # Load model from checkpoint
    model = Wav2Vec2ForCTC.from_pretrained(f"{CHECKPOINT_PREFIX}")

    model.to("cuda")


    # Load optimizer state
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
    optimizer.load_state_dict(torch.load(f"{CHECKPOINT_PREFIX}/optimizer.pt"))

    # Load scheduler state
    scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=100,
        num_training_steps=(NUM_EPOCHS_PER_CHUNK * (iter_index + 1)) * (len(train_dataset) // BATCH_SIZE)
    )
    scheduler.load_state_dict(torch.load(f"{CHECKPOINT_PREFIX}/scheduler.pt"))


❇️ [LOAD INFO] Initializing new tokenizer and processor...
❇️ [LOAD INFO] Starting model training from scratch...


/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at /kaggle/input/wav2vec-demo-model2 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:2175: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


**3.2 Loading tokenizer and processor** <br>
This step is only required for data_collator and won't use for data processing at this step as data is already processed accordingly

In [29]:
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
    "tanmoyio/wav2vec2-large-xlsr-bengali",
    unk_token="[UNK]",
    pad_token="[PAD]",
    word_delimiter_token="|"
)

vocab_dict = tokenizer.vocab

with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [9]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [10]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [11]:
wer_metric = load("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
# # Loading a pre-trained checkpoint
# model = Wav2Vec2ForCTC.from_pretrained(
#     "imrnh/wav2vec_demo_model",
#     ctc_loss_reduction="mean",
#     pad_token_id=processor.tokenizer.pad_token_id,
# )



In [84]:
repo_name = "wav2vec_finetune_bengali_asr"

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=24,
  eval_strategy="steps",
  num_train_epochs=1,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=50,
  eval_steps=50,
  logging_steps=50,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=50,
  save_total_limit=2,
  report_to="none",
)

In [97]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor.feature_extractor,
    optimizers = (optimizer, scheduler)
)

/tmp/ipykernel_35/1112584941.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
if iter_index > 0 and os.path.exists(f"{CHECKPOINT_PREFIX}/trainer_state.json"): # Resume training state if available
    print("❇️ [RESUME TRAINING]")
    trainer.train(resume_from_checkpoint=f"{CHECKPOINT_PREFIX}")
else:
    print("❇️ [STARTING TRAINING FROM SCRATCH]")
    trainer.train()

❇️ [STARTING TRAINING FROM SCRATCH]


In [79]:
trainer.save_model(f"{CHECKPOINT_PREFIX}") # Save pre-trained model

processor.save_pretrained(f"{CHECKPOINT_PREFIX}") # Save pre-trained tokenizer

torch.save(optimizer.state_dict(), f"{CHECKPOINT_PREFIX}/optimizer.pt") # Save optimizer

torch.save(scheduler.state_dict(), f"{CHECKPOINT_PREFIX}/scheduler.pt") # Save scheduler

trainer.state.save_to_json(f"{CHECKPOINT_PREFIX}/trainer_state.json") # Save training state

# Update iteration index
with open(LOAD_CONFIG_PATH, "w") as f:
    f.write(str(iter_index + 1))

print(f"❇️ [LOAD INFO] Finished training on chunk {iter_index}. Ready for next chunk.")
print(f"\nSaved checkpoint to {CHECKPOINT_PREFIX}_{iter_index}")

❇️ [LOAD INFO] Finished training on chunk 16. Ready for next chunk.

Saved checkpoint to /kaggle/working/checkpoints/w2v_16


In [80]:
from huggingface_hub import HfApi
import torch

REPO_NAME = "Anisa206/wav2vec_finetune_bengali_asr"
api = HfApi()

# ✅ Save training artifacts locally
torch.save(optimizer.state_dict(), "optimizer.pt")
torch.save(scheduler.state_dict(), "scheduler.pt")
trainer.state.save_to_json("trainer_state.json")
with open("load.config", "w") as f:
    f.write(str(iter_index + 1))

# ✅ Upload all to Hugging Face under the /w2v directory
for filename in ["optimizer.pt", "scheduler.pt", "trainer_state.json", "load.config"]:
    api.upload_file(
        path_or_fileobj=filename,
        path_in_repo=f"w2v/{filename}",
        repo_id=REPO_NAME
    )

print("✅ Uploaded all training state files to w2v/ on Hugging Face Hub.")


Uploading...:   0%|          | 0.00/756M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Uploading...:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


✅ Uploaded all training state files to w2v/ on Hugging Face Hub.


In [ ]:
import shutil
import os
from IPython.display import FileLink

# Copy all checkpoints + config to a slim directory
shutil.copytree('/kaggle/working/checkpoints', '/kaggle/working/slim_checkpoints', dirs_exist_ok=True)
shutil.copy('/kaggle/working/load.config', '/kaggle/working/slim_checkpoints/load.config')

# Zip the slim checkpoints folder
!zip -r /kaggle/working/slim_checkpoints.zip /kaggle/working/slim_checkpoints > /dev/null

# Create download link
FileLink('/kaggle/working/slim_checkpoints.zip')


In [ ]:
import shutil
from IPython.display import FileLink

# Copy the safetensors file to the root of /kaggle/working
shutil.copy('/kaggle/working/slim_checkpoints/w2v/model.safetensors', '/kaggle/working/model.safetensors')

# Zip just the file
!zip -j /kaggle/working/model_safetensors.zip /kaggle/working/model.safetensors > /dev/null

# Create a download link
FileLink('/kaggle/working/model_safetensors.zip')


In [ ]:
import shutil
from IPython.display import FileLink

# Copy the safetensors file to the root of /kaggle/working
shutil.copy('/kaggle/working/slim_checkpoints/w2v/optimizer.pt', '/kaggle/working/optimizer.pt')

# Zip just the file
!zip -j /kaggle/working/optimizer_pt.zip /kaggle/working/optimizer.pt > /dev/null

# Create a download link
FileLink('/kaggle/working/optimizer_pt.zip')


In [ ]:
import shutil
import os

# Create an "outputs" directory in /kaggle/working/ if it doesn't exist
output_dir = "/kaggle/working/outputs"
os.makedirs(output_dir, exist_ok=True)

# Move the zip file there
shutil.move("/kaggle/working/slim_checkpoints.zip", f"{output_dir}/slim_checkpoints.zip")


In [ ]:
import os
import shutil

dir_path = "/kaggle/working/checkpoints/w2v_0"

if os.path.exists(dir_path) and os.path.isdir(dir_path):
    shutil.rmtree(dir_path)
    print("✅ Directory deleted.")
else:
    print("⚠️ Directory not found.")


In [ ]:
# 00203b850e.flac

# /root/.cache/huggingface/datasets/downloads/extracted/26bb8427288fee2953f209d1a76d10a296b928d87eeee035cf51e56f8508f2fe/asr_bengali/data/00/00203b850e.flac